# JUICE RPWI HF SID5 (PSSR1 surv): L1a QL -- 2023/10/29

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

# os.environ["CDF_LIB"] = "C:\\Program Files\\CDF_Distribution\\cdf39_0-dist\\lib"
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid5_lib as juice_sid5

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
unit_mode = 0                           # 0: sum    1: /Hz
cal_mode = 0                            # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 2.5                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(unit_mode, cal_mode, cal, p_raw_max, p_raw_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder
data_dir = '/Users/user/0-python/JUICE_data/'        # CDF data folder

# *** Ver.2 ***

# 202310 -- SAMPLE
data_name = 'test-CCSDS/sample/cdf/SID05_20231024-0046.cdf'

# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid5.juice_getdata_hf_sid5(cdf)
date = data.epoch[0]
month = date.strftime('%Y%m')
print(data.epoch[0], "-", data.epoch[-1], "[", month, "]")

data.EE = data.EE * 10**(cf/10)
data.EE_dB = 10.0 * np.log10(data.EE)

n_freq = data.EE.shape[1]
n_time = data.EE.shape[0]
n_time0 = n_time-1
print("data size:", data.EE.shape, n_time, n_freq)

In [ ]:
# Mode 
print("[Data Mode]")
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
N_component = 1
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
N_frequency = data.N_freq[0]   # U_selected[0]np.int16(data.B0_step[0] * data.B0_subdiv[0] / data.subdiv_reduction[0])
print("==> Component", N_component, "   Num-frequency", N_frequency, "  Num-sweep:", n_time)

# Data for checks

In [ ]:
Epoch_1d = data.epoch.tolist()
freq_1d  = data.frequency[0].tolist()
freq1  = freq_1d[0:N_frequency]
freq_width_1d  = data.freq_width[0].tolist()
freq_width1  = freq_width_1d[0:N_frequency]
freq_width_2d = data.freq_width.transpose()

EE_2d = data.EE.transpose()
EE_power1 = data.EE[0][0:N_frequency]
EE_power2 = data.EE[n_time0][0:N_frequency]
if unit_mode == 1:
    EE_2d = EE_2d / freq_width_2d / 1000 * data.subdiv_reduction[0]**0.5
    EE_power1 = EE_power1 / freq_width1 / 1000 * data.subdiv_reduction[0]**0.5
    EE_power2 = EE_power2 / freq_width1 / 1000 * data.subdiv_reduction[0]**0.5
    print("frequency width:", freq_width1[0], data.subdiv_reduction[0])
    
f_min = freq_1d[0]
f_max = freq_1d[N_frequency-1]
power_str = juice_cdf.power_label(cal_mode, unit_mode)

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(np.ravel(data.EE), '-r', linewidth=.5)
ax1.set_yscale('log')
ax2.plot(np.ravel(data.frequency), '-r', linewidth=.5)
ax2.plot(np.ravel(data.freq_step*100), '-g', linewidth=.5)
ax2.plot(np.ravel(data.freq_width*100), '-b', linewidth=.5)
ax3.plot(np.ravel(data.epoch), '.')

ax1.set_ylabel('Power [RAW @ ADC]')
ax2.set_ylabel('frequency [kHz]')
ax3.set_ylabel('Date-Time')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax3.set_xlabel(date)

"""
# range: X-axis
xlim=[0, N_frequency]
ax1.set_xlim(xlim)
ax2.set_xlim(xlim)
"""
# range: Y-axis
ylim=[10**p_raw_min, 10**p_raw_max]
ax1.set_ylim(ylim)
ylim=[f_min, f_max]
ax2.set_ylim(ylim)

# Plot
fig.show
png_fname = work_dir+data_name+'_raw.png'
# fig.savefig(png_fname)

## Spec LOG

In [ ]:
# Y-range
p_min0 = p_min
p_max0 = p_max

fig, cal = plt.subplots(figsize=(12, 6))
cal.plot(freq1, EE_power1, '-r', linewidth=.5)
cal.plot(freq1, EE_power2, '-b', linewidth=.5)

cal.set_xlabel('Frequency [kHz]')
cal.set_xscale('log')
cal.set_xlim(f_min, f_max)

cal.set_yscale('log')
plt.ylabel(power_str)
cal.set_ylim(10**p_min0, 10**p_max0)

plt.show
png_fname = work_dir+data_name+'_spec-log.png'
# fig.savefig(png_fname)

## Spec-linear

In [ ]:
# Y-range
p_min0 = p_min
p_max0 = p_max

fig, cal = plt.subplots(figsize=(12, 6))
cal.plot(freq1, EE_power1, '-r', linewidth=.5)
cal.plot(freq1, EE_power2, '-b', linewidth=.5)

cal.set_xlabel('Frequency [kHz]')
cal.set_xlim(f_min, f_max)

cal.set_yscale('log')
plt.ylabel(power_str)
cal.set_ylim(10**p_min0, 10**p_max0)

plt.show
png_fname = work_dir+data_name+'_spec.png'
# fig.savefig(png_fname)

## FT - LOG

In [ ]:
p_min0 = p_min
p_max0 = p_max

fig2d = plt.figure(figsize=[12,3])
ax1 = fig2d.add_subplot(1, 1, 1)

# Y-axis
ax1.set_ylim(f_min, f_max)
ax1.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax1.set_xlabel(date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp1.set_label(power_str)

plt.show()
png_fname = work_dir+data_name+'_FT-log.png'
# fig2d.savefig(png_fname)


## FT - linear

In [ ]:
p_min0 = p_min
p_max0 = p_max

fig2d = plt.figure(figsize=[12,3])
ax1 = fig2d.add_subplot(1, 1, 1)

# Y-axis
ax1.set_ylim(f_min, f_max)
ax1.set_ylabel('f [kHz]')

# X-axis
ax1.set_title('EE')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax1.set_xlabel(date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp1.set_label(power_str)

plt.show()
png_fname = work_dir+data_name+'_FT.png'
# fig2d.savefig(png_fname)